In [5]:
import numpy as np
from scipy.stats import norm

In [20]:
class VanillaOption:
    def __init__(self, K, T, option_type):
        self.K = K
        self.T = T
        self.option_type = option_type
        if self.option_type not in ['Call', 'Put']:
            raise ValueError("option_type must be 'Call' or 'Put'")

    def payoff(self, spot):
        if self.option_type == 'Call':
            return max(spot - self.K, 0)
        else:
            return max(self.K - spot, 0)

In [19]:
class BSModel:
    def __init__(self, S, sigma, r, q):
        self.S = S
        self.sigma = sigma
        self.r = r
        self.q = q

    def mc(self, T, n):
        Z = np.random.randn(n)
        ST = self.S * np.exp((self.r - self.q - 0.5 * self.sigma **2) * T + self.sigma * np.sqrt(T)*Z)
        return ST 

In [18]:
class Pricer:
    def __init__(self, model):
        self.model = model

    def price(self, option):
        S = self.model.S
        sigma = self.model.sigma
        r = self.model.r
        q = self.model.q

        K = option.K
        T = option.T
        option_type = option.option_type

        d1 = (np.log(S/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        if option_type == "Call":
            price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        else:
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

        return price

In [17]:
class PricerMC:
    def __init__(self,model):
        self.model = model

    def price(self, option, n):
        T = option.T
        ST = self.model.mc(T, n)
        payoffs = np.array([option.payoff(s) for s in ST])
        price = np.exp(-self.model.r * T) * np.mean(payoffs)
        return price

In [16]:
bs = BSModel(S=100, sigma = 0.2, r = 0.05, q=0.01)

option = VanillaOption(K=90, T=1, option_type='Call')

pricer = Pricer(model=bs)

print(f"{option.option_type} price : {pricer.price(option):.4f}")

Call price : 15.8938


In [21]:
option = VanillaOption(K = 90, T = 1, option_type='Call')

model = BSModel(S = 100, sigma = 0.2, r = 0.05, q = 0.01)

pricer = PricerMC(model)

prix_mc = pricer.price(option, n = 100000)

print(prix_mc)


15.851036556285258
